#### Loss type
there are different loss type: margin, softplus etc, but not clear yet what exactly they are

#### tail prediction


In [ ]:
from pykeen.models.predict import get_tail_prediction_df

df = get_tail_prediction_df(
    model=result.model,
    head_label="belgium",
    relation_label="locatedin",
    triples_factory=result.training,
    add_novelties=False,
)


#### ranking tier
- The optimistic rank assumes that the true choice is on the first position of all those with equal score.

- The pessimistic rank assumes that the true choice is on the last position of all those with equal score.

- The realistic rank is the mean of the optimistic and the pessimistic rank, and moreover the expected value over all permutations respecting the sort order.

- The non-deterministic rank delegates the decision to the sort algorithm. Thus, the result depends on the internal tie breaking mechanism of the sort algorithm’s implementation.

#### sure about this? 
When early stopping is used during training, it periodically uses the validation set for calculating the loss and evaluation metrics.


### pipeline should not be directly used:
pykeen.pipeline.pipeline() that has no optimization, no early stopping, nor any post-hoc choices using the validation set, the set of known positive triples comprises the training and testing sets. This scenario is very atypical, and regardless, should be augmented with the validation triples to make more comparable to other published results that do not consider this scenario.


#### Using pipeline about validation:
In case the validation triples should not be filtered when evaluating the test dataset, the argument filter_validation_when_testing=False can be passed to either the pykeen.hpo.hpo_pipeline() or pykeen.pipeline.pipeline()

### Example of valuate specific relations only:

In [ ]:
from pykeen.pipeline import pipeline

evaluation_relation_whitelist = {'CtD', 'CpD'}
pipeline_result = pipeline(
    dataset='Hetionet',
    model='RotatE',
    evaluation_relation_whitelist=evaluation_relation_whitelist,
)

### Optimization
use hpo_pipeline for optimization, which may have model specific range or default range

if to set optimization range example one:

In [ ]:
from pykeen.hpo import hpo_pipeline
hpo_pipeline_result = hpo_pipeline(
    dataset='Nations',
    model='TransE',
    model_kwargs_ranges=dict(
        embedding_dim=dict(type=int, low=16, high=256, step=32),
    ),
)

during optimization set number of trails or time out

In [ ]:
from pykeen.hpo import hpo_pipeline
hpo_pipeline_result = hpo_pipeline(
    n_trials=30,  or timeout=60
    dataset='Nations',
    model='TransE',
)

example set fix value during optimization

In [ ]:
from pykeen.hpo import hpo_pipeline
hpo_pipeline_result = hpo_pipeline(
    model='TransE',
    model_kwargs=dict(
        embedding_dim=200,
    ),
    dataset='Nations',
    n_trials=30,
)

early stopping in hpo

In [ ]:
from pykeen.hpo import hpo_pipeline
hpo_pipeline_result = hpo_pipeline(
    n_trials=30,
    dataset='Nations',
    model='TransE',
    stopper='early',
    stopper_kwargs=dict(frequency=5, patience=2, relative_delta=0.002),
)

example of use grid search in optimization--> less efficent 

In [ ]:
from pykeen.hpo import hpo_pipeline
from optuna.samplers import GridSampler
hpo_pipeline_result = hpo_pipeline(
    n_trials=30,
    sampler=GridSampler,
    sampler_kwargs=dict(
        search_space={
            "model.embedding_dim": [32, 64, 128],
            "model.scoring_fct_norm": [1, 2],
            "loss.margin": [1.0],
            "optimizer.lr": [1.0e-03],
            "negative_sampler.num_negs_per_pos": [32],
            "training.num_epochs": [100],
            "training.batch_size": [128],
        },
    ),
    dataset='Nations',
    model='TransE',
)

#### Full setting  

In [ ]:
from pykeen.hpo import hpo_pipeline_from_config
config = {
    'optuna': dict(
        n_trials=30,
    ),
    'pipeline': dict(
        dataset='Nations',
        model='TransE',
        model_kwargs=dict(embedding_dim=20, scoring_fct_norm=1),
        optimizer='SGD',
        optimizer_kwargs=dict(lr=0.01),
        loss='marginranking',
        loss_kwargs=dict(margin=1),
        training_loop='slcwa',
        training_kwargs=dict(num_epochs=100, batch_size=128),
        negative_sampler='basic',
        negative_sampler_kwargs=dict(num_negs_per_pos=1),
        evaluator_kwargs=dict(filtered=True),
        evaluation_kwargs=dict(batch_size=128),
        stopper='early',
        stopper_kwargs=dict(frequency=5, patience=2, relative_delta=0.002),
    )
}
hpo_pipeline_result = hpo_pipeline_from_config(config)

In [ ]:
from pykeen.hpo import hpo_pipeline
hpo_pipeline_result = hpo_pipeline(
    n_trials=30,
    dataset='Nations',
    model='TransE',
    model_kwargs=dict(embedding_dim=20, scoring_fct_norm=1),
    optimizer='SGD',
    optimizer_kwargs=dict(lr=0.01),
    loss='marginranking',
    loss_kwargs=dict(margin=1),
    training_loop='slcwa',
    training_kwargs=dict(num_epochs=100, batch_size=128),
    negative_sampler='basic',
    negative_sampler_kwargs=dict(num_negs_per_pos=1),
    evaluator_kwargs=dict(filtered=True),
    evaluation_kwargs=dict(batch_size=128),
    stopper='early',
    stopper_kwargs=dict(frequency=5, patience=2, relative_delta=0.002),
)

#### Prediction example

In [ ]:


from pykeen.pipeline import pipeline
>>> from pykeen.models import predict
>>> # Run the pipeline
>>> result = pipeline(dataset='Nations', model='RotatE')
>>> # save the model
>>> result.save_to_directory('doctests/nations_rotate')
>>> model = result.model
>>> # Predict tails
>>> predicted_tails_df = predict.get_tail_prediction_df(
...    model, 'brazil', 'intergovorgs', triples_factory=result.training,
... )
>>> # Predict relations
>>> predicted_relations_df = predict.get_relation_prediction_df(
...    model, 'brazil', 'uk', triples_factory=result.training,
... )
>>> # Predict heads
>>> predicted_heads_df = predict.get_head_prediction_df(model, 'conferences', 'brazil', triples_factory=result.training)
>>> # Score all triples (memory intensive)
>>> predictions_df = predict.get_all_prediction_df(model, triples_factory=result.training)
>>> # Score top K triples
>>> top_k_predictions_df = predict.get_all_prediction_df(model, k=150, triples_factory=result.training)
>>> # Score a given list of triples
>>> score_df = predict.predict_triples_df(
...    model=model,
...    triples=[('brazil', 'conferences', 'uk'), ('brazil', 'intergovorgs', 'uk')],
...    triples_factory=result.training,
... )

#### Example of create optimization beyond hpo

In [ ]:


directory = "doctests/ablation/ex05_stopper"
>>> ablation_pipeline(
...    directory=directory,
...    models=["ComplEx"],
...    datasets=["Nations"],
...    losses=["BCEAfterSigmoidLoss", "MarginRankingLoss"],
...    training_loops=["LCWA"],
...    optimizers=["Adam"],
...    stopper = "early",
...    stopper_kwargs = {
...        "frequency": 5,
...        "patience": 20,
...        "relative_delta": 0.002,
...        "metric": "hits@10",
...    },
...    # Fast testing configuration, make bigger in prod
...    epochs=1,
...    n_trials=1,
... )

#### Full example of ablation + hpo example

In [ ]:


from pykeen.ablation import ablation_pipeline
>>> metadata = dict(title="Ablation Study Over Nations for ComplEx.")
>>> models = ["ComplEx"]
>>> datasets = ["Nations"]
>>> losses = ["BCEAfterSigmoidLoss"]
>>> training_loops = ["lcwa"]
>>> optimizers = ["adam"]
>>> create_inverse_triples= [True, False]
>>> stopper = "early"
>>> stopper_kwargs = {
...   "frequency": 5,
...   "patience": 20,
...   "relative_delta": 0.002,
...   "metric": "hits@10",
... }

# Define HPO ranges
>>> model_to_model_kwargs_ranges = {
...   "ComplEx": {
...       "embedding_dim": {
...           "type": "int",
...           "low": 4,
...           "high": 6,
...           "scale": "power_two"
...       }
...   }
... }

>>> model_to_training_loop_to_training_kwargs = {
...   "ComplEx": {
...       "lcwa": {
...           "num_epochs": 50
...       }
...   }
... }

>>> model_to_training_loop_to_training_kwargs_ranges= {
...   "ComplEx": {
...       "lcwa": {
...           "label_smoothing": {
...               "type": "float",
...               "low": 0.001,
...              "high": 1.0,
...               "scale": "log"
...           },
...           "batch_size": {
...               "type": "int",
...               "low": 7,
...               "high": 9,
...               "scale": "power_two"
...           }
...       }
...   }
... }


>>> model_to_optimizer_to_optimizer_kwargs_ranges= {
...   "ComplEx": {
...       "adam": {
...           "lr": {
...               "type": "float",
...               "low": 0.001,
...               "high": 0.1,
...               "scale": "log"
...           }
...       }
...   }
... }

# Run ablation experiment
>>> ablation_pipeline(
...   models=models,
...   datasets=datasets,
...   losses=losses,
...   training_loops=training_loops,
...   optimizers=optimizers,
...   model_to_model_kwargs_ranges=model_to_model_kwargs_ranges,
...   model_to_training_loop_to_training_kwargs=model_to_training_loop_to_training_kwargs,
...   model_to_optimizer_to_optimizer_kwargs_ranges=model_to_optimizer_to_optimizer_kwargs_ranges,
...   directory="doctests/ablation/ex6",
...   best_replicates=5,
...   n_trials=2,
...   timeout=300,
...   metric="hits@10",
...   direction="maximize",
...   sampler="random",
...   pruner="nop",
... )


#### Use transformer label:


In [ ]:

from pykeen.pipeline import pipeline
from pykeen.datasets import get_dataset
from pykeen.nn.representation import EmbeddingSpecification, LabelBasedTransformerRepresentation
from pykeen.models import ERModel

dataset = get_dataset(dataset="nations")
entity_representations = LabelBasedTransformerRepresentation.from_triples_factory(
    triples_factory=dataset.training,
)
result = pipeline(
    dataset=dataset,
    model=ERModel,
    model_kwargs=dict(
        interaction="ermlpe",
        interaction_kwargs=dict(
            embedding_dim=entity_representations.embedding_dim,
        ),
        entity_representations=entity_representations,
        relation_representations=EmbeddingSpecification(
            shape=entity_representations.shape,
        ),
    ),
    training_kwargs=dict(
        num_epochs=1,
    ),
)
model = result.model
